In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [2]:
import gzip
import pickle
from utils.util import *
from train_test import *
import sys
import os
import argparse
from parse_config import ConfigParser

In [3]:
def load_compressed_pickle(filename):
    with gzip.open(filename, 'rb') as f:
        obj = pickle.load(f)
    return obj

In [4]:
restored_data = load_compressed_pickle('/datasets/mind_data/data_dict_compressed.pickle')

In [5]:
user_history=restored_data["user_history"]
entity_embedding=restored_data["entity_embedding"]
relation_embedding=restored_data["relation_embedding"]
entity_adj=restored_data["entity_adj"]
relation_adj=restored_data["relation_adj"]
news_feature=restored_data["news_feature"]
max_entity_freq=restored_data["max_entity_freq"]
max_entity_pos=restored_data["max_entity_pos"]
max_entity_type=restored_data["max_entity_type"]
train_data=restored_data["train_data"]
dev_data=restored_data["dev_data"]
vert_train=restored_data["vert_train"]
vert_test=restored_data["vert_test"]
pop_train=restored_data["pop_train"]
pop_test=restored_data["pop_test"]
item2item_train=restored_data["item2item_train"]
item2item_test=restored_data["item2item_test"]

In [6]:
parser = argparse.ArgumentParser(description='KRED')
parser.add_argument('-f')
parser.add_argument('-c', '--config', default="./config.json", type=str,
                    help='config file path (default: None)')
parser.add_argument('-r', '--resume', default=None, type=str,
                    help='path to latest checkpoint (default: None)')
parser.add_argument('-d', '--device', default=None, type=str,
                    help='indices of GPUs to enable (default: all)')
config = ConfigParser.from_args(parser)

In [7]:
epochs = 5
batch_size = 64
train_type = "single_task"
task = "user2item"

config['trainer']['epochs'] = epochs
config['data_loader']['batch_size'] = batch_size
config['trainer']['training_type'] = train_type
config['trainer']['task'] = task

In [8]:
if config['trainer']['training_type']  == "multi-task":
    data = user_history, entity_embedding, relation_embedding, entity_adj, relation_adj, news_feature, max_entity_freq, max_entity_pos, max_entity_type, train_data, dev_data, vert_train, vert_test, pop_train, pop_test, item2item_train, item2item_test
elif config['trainer']['task'] == "user2item":
    data = user_history, entity_embedding, relation_embedding, entity_adj, relation_adj, news_feature, max_entity_freq, max_entity_pos, max_entity_type, train_data, dev_data
elif config['trainer']['task'] == "item2item":
    data =  user_history, entity_embedding, relation_embedding, entity_adj, relation_adj, news_feature, max_entity_freq, max_entity_pos, max_entity_type, item2item_train, item2item_test
elif config['trainer']['task'] == "vert_classify":
    data = user_history, entity_embedding, relation_embedding, entity_adj, relation_adj, news_feature, max_entity_freq, max_entity_pos, max_entity_type, vert_train, vert_test
elif config['trainer']['task'] == "pop_predict":
    data = user_history, entity_embedding, relation_embedding, entity_adj, relation_adj, news_feature, max_entity_freq, max_entity_pos, max_entity_type, pop_train, pop_test

In [9]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


In [10]:
#Single Task Training

In [11]:
user_history_dict, entity_embedding, relation_embedding, entity_adj, relation_adj, doc_feature_dict, entity_num, position_num, type_num, train_data, test_data = data

In [12]:
train_data_u2i = NewsDataset(train_data)

In [13]:
train_sampler_u2i = RandomSampler(train_data_u2i)

In [14]:
train_dataloader_u2i = DataLoader(train_data_u2i, sampler=train_sampler_u2i,
                                          batch_size=config['data_loader']['batch_size'],
                                          collate_fn=my_collate_fn, pin_memory=False)

In [15]:
criterion = Softmax_BCELoss(config)

In [16]:
train_data_loader = train_dataloader_u2i

In [17]:
device, deviceids = prepare_device(config['n_gpu'])

In [18]:
deviceids

[0]

In [19]:
model = KREDModel(config, user_history_dict, doc_feature_dict, entity_embedding, relation_embedding, entity_adj,relation_adj, entity_num, position_num, type_num).cuda()

In [20]:
optimizer = optim.Adam(model.parameters(), lr=config['optimizer']['lr'], weight_decay=0)

In [21]:
trainer = Trainer(config, model, criterion, optimizer, device, train_data_loader, data[-1])

In [22]:
trainer.train()

model training
0
1


/notebooks/model/News_embedding.py:129: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  entity_num_embedding = self.entity_num_embeddings(torch.tensor(entity_nums).cuda())


2
3


../aten/src/ATen/native/cuda/Indexing.cu:975: indexSelectLargeIndex: block: [367,0,0], thread: [88,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:975: indexSelectLargeIndex: block: [367,0,0], thread: [89,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:975: indexSelectLargeIndex: block: [367,0,0], thread: [90,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:975: indexSelectLargeIndex: block: [367,0,0], thread: [91,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:975: indexSelectLargeIndex: block: [367,0,0], thread: [92,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:975: indexSelectLargeIndex: block: [367,0,0], thread: [93,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:975: indexSelectLargeIndex: block: [367,0,0], t

RuntimeError: CUDA error: device-side assert triggered

In [23]:
from model.News_embedding import News_embedding

In [24]:
news_232 = News_embedding(config, doc_feature_dict, entity_embedding, relation_embedding, entity_adj, relation_adj, entity_num, position_num, type_num)

In [25]:
news_id = [['N48304', 'N19494', 'N29177', 'N13427', 'N56613', 'N548', 'N12889', 'N2994', 'N19829', 'N5123', 'N3346', 'N56586', 'N2665', 'N51061', 'N23817', 'N50454', 'N8391', 'N21623', 'N43295', 'N45589'], ['N29438', 'N29438', 'N62931', 'N64826', 'N58226', 'N7714', 'N27448', 'N15087', 'N60206', 'N19620', 'N22328', 'N2745', 'N39048', 'N6233', 'N9680', 'N2229', 'N13836', 'N19098', 'N60050', 'N57737'], ['N29177', 'N1150', 'N46483', 'N50454', 'N42989', 'N35009', 'N55846', 'N53387', 'N23311', 'N8129', 'N30344', 'N23652', 'N6385', 'N44251', 'N60200', 'N45087', 'N6886', 'N61763', 'N20473', 'N52088'], ['N17401', 'N5227', 'N5227', 'N24094', 'N28833', 'N21623', 'N55805', 'N50846', 'N49146', 'N18036', 'N37550', 'N31318', 'N33835', 'N56514', 'N59053', 'N32312', 'N9647', 'N22063', 'N30353', 'N40429'], ['N29177', 'N49333', 'N30170', 'N58646', 'N26642', 'N15122', 'N17854', 'N53052', 'N46392', 'N42137', 'N62168', 'N9798', 'N32312', 'N64955', 'N26500', 'N8952', 'N31801', 'N38182', 'N14742', 'N64273'], ['N38689', 'N45286', 'N9445', 'N7429', 'N8391', 'N50330', 'N27416', 'N29339', 'N54807', 'N15924', 'N9791', 'N61948', 'N12907', 'N60080', 'N363', 'N63405', 'N6290', 'N57537', 'N36653', 'N51724'], ['N29438', 'N29438', 'N62931', 'N64826', 'N58226', 'N7714', 'N27448', 'N15087', 'N60206', 'N19620', 'N22328', 'N2745', 'N39048', 'N6233', 'N9680', 'N2229', 'N13836', 'N19098', 'N60050', 'N57737'], ['N41089', 'N37942', 'N13137', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0'], ['N29177', 'N16233', 'N10235', 'N27911', 'N29086', 'N35922', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0'], ['N36676', 'N29316', 'N14374', 'N44957', 'N61704', 'N64208', 'N63332', 'N40517', 'N10270', 'N55856', 'N36236', 'N8422', 'N4866', 'N39627', 'N5252', 'N64187', 'N55856', 'N16344', 'N32203', 'N60033'], ['N1150', 'N31900', 'N55846', 'N46039', 'N59704', 'N22161', 'N40944', 'N42620', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0'], ['N26479', 'N40417', 'N50866', 'N44862', 'N38876', 'N63405', 'N21839', 'N46709', 'N34572', 'N55987', 'N38645', 'N54827', 'N14785', 'N23094', 'N51897', 'N16655', 'N409', 'N17161', 'N11020', 'N9732'], ['N33358', 'N12907', 'N59496', 'N53052', 'N53531', 'N43843', 'N45794', 'N501', 'N26109', 'N44396', 'N19293', 'N54625', 'N55043', 'N24298', 'N55051', 'N11005', 'N306', 'N4777', 'N43142', 'N60702'], ['N46063', 'N15605', 'N56586', 'N44559', 'N16233', 'N33969', 'N49481', 'N16292', 'N63499', 'N619', 'N39511', 'N26026', 'N4526', 'N14452', 'N60491', 'N33096', 'N19716', 'N34592', 'N35856', 'N14647'], ['N53033', 'N9120', 'N10535', 'N477', 'N18355', 'N48615', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0'], ['N6023', 'N27608', 'N619', 'N306', 'N52840', 'N4607', 'N42992', 'N48742', 'N64273', 'N35671', 'N7364', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0'], ['N30160', 'N41375', 'N42620', 'N30765', 'N24075', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0'], ['N5227', 'N61018', 'N14102', 'N39376', 'N33920', 'N29276', 'N54883', 'N52396', 'N15672', 'N55024', 'N26179', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0'], ['N46392', 'N21136', 'N40201', 'N33096', 'N6616', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0'], ['N50513', 'N47170', 'N34206', 'N56514', 'N50069', 'N5978', 'N56148', 'N31801', 'N61661', 'N11512', 'N35710', 'N20547', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0'], ['N16215', 'N42458', 'N8215', 'N51706', 'N20121', 'N53642', 'N8314', 'N8010', 'N21977', 'N30662', 'N11894', 'N16071', 'N24651', 'N33976', 'N477', 'N53880', 'N12676', 'N56240', 'N34394', 'N55266'], ['N31927', 'N56630', 'N37811', 'N46868', 'N32027', 'N58744', 'N35560', 'N56753', 'N47173', 'N31801', 'N29269', 'N32852', 'N4985', 'N48692', 'N52840', 'N8448', 'N51503', 'N14761', 'N685', 'N3433'], ['N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0'], ['N9803', 'N17007', 'N12680', 'N9803', 'N6952', 'N18962', 'N33096', 'N46811', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0'], ['N54095', 'N55189', 'N872', 'N56663', 'N42973', 'N50578', 'N16233', 'N53074', 'N53369', 'N24691', 'N44361', 'N56778', 'N7422', 'N9226', 'N45794', 'N9120', 'N40809', 'N3471', 'N30748', 'N11273'], ['N23571', 'N31739', 'N3500', 'N29088', 'N34694', 'N836', 'N62746', 'N7004', 'N52063', 'N7586', 'N59359', 'N1398', 'N18951', 'N90', 'N25078', 'N44867', 'N0', 'N0', 'N0', 'N0'], ['N11461', 'N43954', 'N12900', 'N29597', 'N11282', 'N46337', 'N16390', 'N22479', 'N1569', 'N21553', 'N63092', 'N32942', 'N4345', 'N30965', 'N57265', 'N20148', 'N18881', 'N10059', 'N33358', 'N6167'], ['N26294', 'N3676', 'N22435', 'N45954', 'N55743', 'N27120', 'N12349', 'N8908', 'N37281', 'N15926', 'N18760', 'N64838', 'N26500', 'N63656', 'N60314', 'N4190', 'N20760', 'N29867', 'N8024', 'N60585'], ['N6233', 'N26176', 'N54512', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0'], ['N26642', 'N58264', 'N40716', 'N22397', 'N42528', 'N2309', 'N55556', 'N60702', 'N20575', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0'], ['N30450', 'N19809', 'N13137', 'N24298', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0'], ['N17799', 'N50126', 'N60184', 'N62058', 'N26009', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0'], ['N4166', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0'], ['N22279', 'N51591', 'N7432', 'N37048', 'N60894', 'N17587', 'N7432', 'N15446', 'N50155', 'N56426', 'N42421', 'N42989', 'N10340', 'N60882', 'N55108', 'N20609', 'N23818', 'N58226', 'N38118', 'N59546'], ['N59590', 'N11701', 'N28818', 'N49745', 'N15034', 'N56742', 'N58715', 'N9277', 'N9824', 'N34923', 'N57491', 'N4593', 'N40716', 'N40239', 'N23653', 'N4697', 'N30765', 'N6103', 'N35560', 'N24488'], ['N56253', 'N700', 'N59496', 'N55743', 'N51385', 'N52536', 'N64631', 'N48715', 'N46795', 'N29757', 'N50710', 'N62853', 'N58837', 'N30612', 'N59143', 'N0', 'N0', 'N0', 'N0', 'N0'], ['N29177', 'N55846', 'N11917', 'N13008', 'N10059', 'N24691', 'N1825', 'N5978', 'N26176', 'N47847', 'N45535', 'N51089', 'N27526', 'N46818', 'N14417', 'N50710', 'N8148', 'N0', 'N0', 'N0'], ['N49745', 'N32483', 'N60050', 'N57737', 'N4020', 'N13023', 'N6023', 'N33276', 'N42994', 'N6197', 'N45794', 'N46392', 'N45291', 'N39867', 'N53033', 'N33304', 'N61388', 'N9803', 'N58090', 'N10803'], ['N64716', 'N17401', 'N12900', 'N36133', 'N29669', 'N29177', 'N62429', 'N44251', 'N44579', 'N33454', 'N6233', 'N39117', 'N33969', 'N5290', 'N8845', 'N27911', 'N33276', 'N34172', 'N27395', 'N18094'], ['N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0'], ['N37739', 'N12291', 'N3500', 'N2203', 'N16233', 'N22851', 'N51706', 'N60050', 'N11163', 'N31927', 'N35009', 'N50147', 'N54643', 'N49057', 'N59156', 'N15253', 'N8845', 'N1959', 'N62791', 'N54936'], ['N22570', 'N4643', 'N29177', 'N56586', 'N3046', 'N50292', 'N356', 'N52116', 'N44292', 'N36759', 'N7004', 'N41899', 'N10896', 'N20483', 'N16292', 'N51163', 'N46392', 'N14947', 'N63411', 'N64554'], ['N14454', 'N51073', 'N60303', 'N60615', 'N29526', 'N33513', 'N56586', 'N59004', 'N45286', 'N11291', 'N54095', 'N24138', 'N31391', 'N61347', 'N61217', 'N62084', 'N8443', 'N61089', 'N32892', 'N60313'], ['N42458', 'N38299', 'N31801', 'N60300', 'N38105', 'N19678', 'N49040', 'N23009', 'N59451', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0'], ['N37211', 'N35897', 'N7201', 'N29177', 'N1150', 'N36517', 'N7442', 'N10202', 'N64408', 'N49745', 'N43187', 'N53234', 'N51814', 'N51692', 'N42373', 'N16715', 'N57196', 'N35452', 'N55922', 'N7508'], ['N27448', 'N19638', 'N30662', 'N57971', 'N28053', 'N60699', 'N60383', 'N24298', 'N56753', 'N34797', 'N19347', 'N40944', 'N64474', 'N51503', 'N14761', 'N4607', 'N22816', 'N35671', 'N0', 'N0'], ['N38701', 'N11348', 'N10602', 'N14975', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0'], ['N35290', 'N17050', 'N53306', 'N36514', 'N37814', 'N58232', 'N14452', 'N14108', 'N55556', 'N60585', 'N14689', 'N39733', 'N64474', 'N21258', 'N63033', 'N16110', 'N37168', 'N19511', 'N59574', 'N10359'], ['N14761', 'N46846', 'N40509', 'N21220', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0'], ['N19381', 'N63276', 'N37120', 'N65201', 'N63930', 'N42458', 'N16695', 'N61961', 'N20145', 'N14644', 'N14644', 'N3875', 'N11382', 'N57255', 'N65169', 'N15749', 'N49289', 'N25351', 'N59289', 'N2306'], ['N10865', 'N787', 'N60290', 'N52403', 'N18714', 'N29361', 'N3940', 'N29227', 'N59285', 'N34069', 'N32377', 'N12215', 'N64980', 'N62539', 'N40207', 'N38105', 'N7057', 'N28901', 'N36615', 'N11654'], ['N19494', 'N26136', 'N47069', 'N2580', 'N46567', 'N18760', 'N49088', 'N34669', 'N55667', 'N52116', 'N39117', 'N59676', 'N5310', 'N17432', 'N42760', 'N61018', 'N12367', 'N37942', 'N4415', 'N53531'], ['N49946', 'N12900', 'N8548', 'N39662', 'N719', 'N32893', 'N986', 'N36625', 'N18247', 'N64208', 'N44361', 'N6583', 'N16260', 'N47671', 'N7014', 'N26824', 'N33276', 'N15796', 'N27154', 'N9293'], ['N11177', 'N31801', 'N64273', 'N19773', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0'], ['N11703', 'N3501', 'N17587', 'N37739', 'N59283', 'N51591', 'N6523', 'N37878', 'N5163', 'N49436', 'N64050', 'N55189', 'N10059', 'N53879', 'N47765', 'N33969', 'N54581', 'N27644', 'N4198', 'N39118'], ['N53781', 'N55148', 'N39888', 'N48332', 'N65039', 'N63248', 'N59546', 'N41229', 'N41219', 'N1633', 'N43749', 'N62429', 'N63650', 'N10629', 'N32223', 'N38118', 'N18743', 'N51998', 'N24721', 'N37976'], ['N64355', 'N46802', 'N55148', 'N51591', 'N27954', 'N4121', 'N37169', 'N15539', 'N32095', 'N21298', 'N65232', 'N40639', 'N27382', 'N54704', 'N12110', 'N31699', 'N2142', 'N50966', 'N9890', 'N45331'], ['N11461', 'N36211', 'N3499', 'N24486', 'N3501', 'N29426', 'N21120', 'N26875', 'N10426', 'N41997', 'N11382', 'N43187', 'N58235', 'N60979', 'N19125', 'N50033', 'N9803', 'N38821', 'N18657', 'N18873'], ['N44251', 'N55189', 'N38963', 'N3454', 'N51112', 'N3926', 'N38659', 'N24721', 'N32089', 'N12845', 'N9804', 'N55043', 'N22161', 'N10835', 'N58580', 'N13374', 'N26026', 'N4526', 'N16380', 'N27818'], ['N17587', 'N51591', 'N58764', 'N59798', 'N61719', 'N26303', 'N30662', 'N34206', 'N31975', 'N36934', 'N36488', 'N11101', 'N13707', 'N7649', 'N59027', 'N41471', 'N54827', 'N40716', 'N50744', 'N12709'], ['N41045', 'N14842', 'N28000', 'N11346', 'N32942', 'N4978', 'N16392', 'N36217', 'N33358', 'N46770', 'N35897', 'N4978', 'N54496', 'N42842', 'N59767', 'N31177', 'N4562', 'N27907', 'N10818', 'N27907'], ['N60046', 'N22455', 'N32874', 'N43142', 'N53872', 'N53422', 'N14761', 'N46818', 'N57140', 'N18085', 'N6225', 'N22816', 'N22300', 'N7938', 'N47520', 'N0', 'N0', 'N0', 'N0', 'N0'], ['N63236', 'N20145', 'N49903', 'N9120', 'N9743', 'N63767', 'N54179', 'N55588', 'N49364', 'N13138', 'N22273', 'N11273', 'N54536', 'N37509', 'N6727', 'N44440', 'N27993', 'N19665', 'N5791', 'N37250'], ['N29177', 'N60209', 'N9226', 'N40322', 'N9120', 'N12306', 'N42620', 'N42620', 'N6616', 'N48699', 'N42128', 'N32004', 'N53017', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0', 'N0']]


In [26]:
entities = news_232.get_entities_ids(news_id)

In [27]:
entities = news_232.get_entities_ids(news_id)
entities

[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [65208, 25454, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [136331, 92910, 78207, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [111514, 243358, 172, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [259277, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [403925, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [50257,
   439844,
   378227,
   46374,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  [1888658, 77885, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [1212, 5288, 100095, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [4123,
   758266,
   758267,
   791229,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  [720969, 0, 0, 0

In [28]:
entities = news_232.get_entities_ids(news_id)

In [29]:
kgat2232 = news_232.kgat

In [30]:
neighbor_entities, neighbor_relations = kgat2232.get_neighbors(entities)

0


In [31]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [32]:
kgat2232.entity_embedding.cuda()

RuntimeError: CUDA error: device-side assert triggered

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0145, -0.1069,  0.0246,  ..., -0.0476, -0.0190, -0.0804],
        [-0.1405, -0.1619, -0.0266,  ...,  0.0140, -0.0089, -0.1726],
        ...,
        [ 0.0159,  0.0218,  0.0043,  ..., -0.0231, -0.0946, -0.0183],
        [-0.0092,  0.0011,  0.0450,  ..., -0.0621, -0.0775, -0.0038],
        [ 0.0264,  0.0190,  0.0385,  ...,     nan,     nan,     nan]],
       dtype=torch.float16)

In [41]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [42]:
torch.tensor(kgat2232.entity_embedding).cuda()

/tmp/ipykernel_76/2387496285.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(kgat2232.entity_embedding).cuda()


RuntimeError: CUDA error: device-side assert triggered

In [ ]:
entity_embedding_lookup = nn.Embedding.from_pretrained(kgat2232.entity_embedding)
relation_embedding_lookup = nn.Embedding.from_pretrained(kgat2232.relation_embedding)

In [70]:
neighbor_relation_embedding = relation_embedding_lookup(torch.tensor(neighbor_relations))

In [71]:
entity_embedding = entity_embedding_lookup(torch.tensor(entities))

In [44]:
print(f'{len(entity_nums)}, {len(entity_nums[0])}, {len(entity_nums[0][0])}')


64, 20, 20


In [37]:
entity_num_embedding = news_232.get_entity_num_embedding(entity_nums)


RuntimeError: CUDA error: device-side assert triggered